# Getting Tweets From Twitter Streaming API

In this notebook we will use the library Tweepy to access Twitter API. Tweepy is a python library that facilitate accessing the API. To get started you have to apply for a developer's account to get keys and access token. For more details consult this page: https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25

In [81]:
import tweepy

In [82]:
twitter_keys = {
    'consumer_key':        '',
    'consumer_secret':     '',
    'access_token_key':    '',
    'access_token_secret': ''
}

Let's try a simple experiment, we will import the tweets from my(your) timeline...

In [83]:
#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth)

#Make call on home timeline, print each tweets text
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

Drought and frosts in Brazil may be starting a major price correction that shifts the coffee market upwards for yea… https://t.co/1gymoEsTUx
RT @trudymorgancole: #hfxfullycommitted was an amazing show by an incredible performer -- if you have a chance to see it before it closes,…
RT @DowntownHalifax: We've teamed up again with @PrismaticArts on another giveaway! 2 lucky winners will receive 2 tickets to any performan…
Pinterest has hired Martin Galvin, GroupM’s U.K. commercial strategy director, to lead its commercial partnerships… https://t.co/vgSgIAObU0
Holiday savings for customers start now! Check out how we're kicking off Black Friday-worthy deals, in addition to… https://t.co/oZyL1no79h
RT @TwitCoast: Breton Lalama began preparing for his latest role—a starring turn in @NeptuneTheatre's season-opening play Fully Committed,…
RT @Forbes: Here's everything we know so far about the Apple AirPods 3: https://t.co/odYSJ6BWUt https://t.co/74k15ZsChb
هذا العالم سيفنى بسبب جرعة زائدة من ال

Now let's see how a single tweet looks like..

In [84]:
public_tweets[0]

Status(_api=<tweepy.api.API object at 0x7fbee6b8f990>, _json={'created_at': 'Mon Oct 04 14:15:53 +0000 2021', 'id': 1445029912990167045, 'id_str': '1445029912990167045', 'text': 'Drought and frosts in Brazil may be starting a major price correction that shifts the coffee market upwards for yea… https://t.co/1gymoEsTUx', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/1gymoEsTUx', 'expanded_url': 'https://twitter.com/i/web/status/1445029912990167045', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2735591, 'id_str': '2735591', 'name': 'Fast Company', 'screen_name': 'FastCompany', 'location': 'New York, NY', 'description': 'Inspiring readers to th

Oh..that's hard to examine..

For better readability try this..

In [85]:
import json 

status = public_tweets[1]

#convert to string
json_str = json.dumps(status._json)

#deserialise string into python object
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Mon Oct 04 14:15:42 +0000 2021",
    "entities": {
        "hashtags": [
            {
                "indices": [
                    21,
                    39
                ],
                "text": "hfxfullycommitted"
            }
        ],
        "symbols": [],
        "urls": [],
        "user_mentions": [
            {
                "id": 45576341,
                "id_str": "45576341",
                "indices": [
                    3,
                    19
                ],
                "name": "Trudy Morgan-Cole",
                "screen_name": "trudymorgancole"
            }
        ]
    },
    "favorite_count": 0,
    "favorited": false,
    "geo": null,
    "id": 1445029867150577667,
    "id_str": "1445029867150577667",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_s

This also help you understand the structure of a tweet, so you can decide which fields you need in your application.

## Getting More Data

We've seen a simple example, now we want to get real data and store them in a file for later processing. Define a file and open it for writing 'w', or append 'a'.

In [86]:
fName = 'sampledata.txt' # We'll store the tweets in a text file.
f = open(fName, 'w')

### Next, we have to import few more libraries.

**StreamListener**: the on_data method of Tweepy’s StreamListener passes data from statuses to the on_status method. The class StdOutListener inheriting from StreamListener and overriding on_status. <br>
**OAuthHandler**: as the name implies, this object handles authentication. Into this we pass our consumer key and secret. <br>
**Stream**: establishes a streaming session and routes messages to StreamListener instance.  <br>
**Time**: provides various time-related functions. <br>
**jsonpickle**: is a Python library for serialization and deserialization of complex Python objects to and from JSON. JSON is a format that encodes objects in a string. Serialization aims to convert an object into that string, and deserialization is its inverse operation (convert string back to object).

In [87]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import jsonpickle

Here, we create a class with methods to handle the streaming API, and errors

In [88]:
class StdOutListener(StreamListener):

    """ A listener handles tweets that are the received from the stream.
    """
    def __init__(self):
        super().__init__()
        self.max_tweets = 500
        self.tweet_count = 0
    
    def on_status(self, status):
        f.write(jsonpickle.encode(status._json, unpicklable=False) + '\n')
        print (status.user.id, '-', status.text) 
        self.tweet_count+=1
        if(self.tweet_count==self.max_tweets):
            print("completed")
            return False
        
    def on_error(self, status_code):
      if status_code == 420:
        #returning False in on_data disconnects the stream
        return False

    def on_timeout(self):
        print (sys.stderr, 'Timeout...')
        # sys: System-specific parameters and functions, stderr: error returned
        return True # Don't kill the stream

You can optionally add keywords to filter the stream, otherwise you will get random tweets.¶

In [89]:
filterkw = ["موسم الرياض", "كورونا"]

In [90]:
l = StdOutListener()
auth = OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
stream = Stream(auth, l)
stream.filter(track = filterkw)

275577537 - #إسرائيل.. وفيات كورونا بين رافضي التطعيم تسجل "أرقاما دراماتيكية" https://t.co/DYHbKvj7JI 
#السياسة
1427417397636603911 - RT @AJELNEWS24: عاجل 🔴 
.

.
تركي آل الشيخ يعلن رسمياً إطلاق دعاية موسم الرياض .. تخيل أكثر .
.

.

#موسم_الرياض

https://t.co/F056lr8H8F
1011344312200781824 - @gorgeous4ew ابي مبارات عالمية بين النساء …. لان هذا موسم الرياض
1618763372 - RT @emadmonshi: بمناسبة اطلاق موسم الرياض في نسخته الثانية، تدعوكم #الجمعية_السعودية_للسياحة

لمحاضرة:
قراءة علمية في مشاركة رئيس مجلس إدار…
1420722682618843137 - اللهم عافنا
1421724528154234884 - #الكتلة_الصدرية_عراقية

 لأنهم لبوا نداء الوطن عندما عصفت به جائحة كورونا ، فكانوا بلسماً لجراح المصابين ... فشكرا لهم ..
1437416824086347781 - RT @AlshrabiAsem: فذلك لِمَن كَلَّف نفسه لقراءتها بَجَدٍّ وصِدقٍ من غير استهزاءٍ فيجد فيها الشّفاء من فايروس كورونا من غير لقاحٍ ولا دواءٍ.…
1003738282545557505 - RT @SufwatHaddadin: الشكر العميق للأهل و العزوة الأردنيين و العرب الذين غمروني باهتمامهم و كلماتهم التي كانت في مجملها أكثر

1276517634310275072 - RT @t3abir: ..

المملكة ثانيا عالميا في التعافي من كورونا وفق مؤشر نيكاي ..
وطن التطور والتقدم ..

#تعابير #الهلال #هي_لنا_دار #اكشن_مع_ولي…
1355934612149694464 - بعدين خير تجيني كورونا الحين
2273167643 - RT @Hamid_ALhuthali: @badralialotibi1 https://t.co/MRn9fRLuTF
932334680967401473 - RT @gorgeous4ew: عاجل:

ظهور الأسطورة ميسي في إعلان موسم الرياض 😍🇸🇦! https://t.co/tAVuafmXDL
1260359653 - @Riiim2__ من بدت كورونا 😂😂✋🏼
1252282524736409602 - RT @soalba1: #موسم_الرياض 

سعيده مره احس الحياة رجعت مثل قبل كورونا 😍💗💗🤍
1436636575807459332 - RT @fa__tim__a: #الكتلة_الصدرية_عراقية

 لأنهم لبوا نداء الوطن عندما عصفت به جائحة كورونا ، فكانوا بلسماً لجراح المصابين ... فشكرا لهم ..
1350341482646036481 - RT @Team_10_Messi: الأسطورة ميسي في إعلان موسم الرياض 😍 https://t.co/9SvZ3QXHtb
39086234 - ما زهقتوا بعد 🙄
14231026 - ابتداءً من اليوم.. سيعفى المسافرون القادمون من دولة قطر من الحجر الصحي عند الوصول وذلك بعد أعتماد لقاحات كورونا الم… https://t.co/GRpMY6UBB6
132622294763220992

995734722 - RT @AlwatanNews24: تركي آل الشيخ يعلن رسمياً إطلاق دعاية موسم الرياض .. تخيل أكثر .
.
.

#موسم_الرياض

-

https://t.co/TO5b8VTTHq
1219601114733195272 - #المهدي_اليماني_ناصر_محمد
فَتَجِدوا حقيقة مَلَف القضيّة كاملة في شأن فايروس كورونا مِن البداية إلى النّهاية.… https://t.co/5jjdXft48S
630680756 - RT @mohamedfarrakha: مدرسة الشويفات بدريم لاند 
أي إجراءات إحترازية أتخذتوها ليصاب أطفال صف كامل تقريبا ب كورونا
1142654578988597248 - @3m__1 @6gAqWwTvUe3QgFW @osaamah11 علمو صديقته ان موسم الرياض بيبدأ ان شاء الله 20 اكتوبر 
تمشيه وفله حجاج وسعة صدر
1371956826149953543 - مابتجي مباراة باريس في موسم الرياض الا تاليسكا وبيريرا فقعوا
235069735 - RT @gorgeous4ew: عاجل:

ظهور الأسطورة ميسي في إعلان موسم الرياض 😍🇸🇦! https://t.co/tAVuafmXDL
473194793 - #السعودية تتصدر المرتبة الثانية عالميًا من بين 120 دولة في التعافي من كورونا | مجلة سيدتي https://t.co/3xqaIEyjoH
922665478471999488 - RT @AJELNEWS24: عاجل 🔴 
.

.
تركي آل الشيخ يعلن رسمياً إطلاق دعاية موسم الرياض .. تخيل أكثر .
.

.

#موس

1302291408037072898 - RT @wqvz24: تم ذكر ان هناك 76 حفله غنائيه عالميه و عربيه و ذكروا ان نسبه 2٪ من موسم الرياض عباره عن حفلات غنائيه و قام حساب مشاريع السعوديه…
2535640027 - RT @SaudiNews50: عاجل:

رئيس هيئة الترفيه:

4 مناطق في موسم الرياض سيكون الدخول فيها مجاناً.

#موسم_الرياض
1046603466574057473 - RT @FCBW_M30: اعلان موسم الرياض برعاية الاسطورة ميسي😍😍😍😍😍 https://t.co/Cs1M2zm6RP
1429487686935527428 - @SaudiNews50 *فحص كارونا PCR🚩*
*فحص كارونا PCR🚩*
*✅فحص كارونا للسفر✅*
*متاح تقرير طبي كورونا للسفر*
*معتمد للسفر من… https://t.co/befaEtgrHd
756154992705232896 - RT @SaudiMOH: #الصحة⁩ تعلن عن تسجيل (53) حالة إصابة جديدة بفيروس كورونا (كوفيد-19)، وتسجيل (3) حالات وفيات رحمهم الله، وتسجيل (40) حالة تعا…
559249936 - @GonewithTheWin7 @AnwarAboalela مصيره يتخفض لذلك هذا النفط لا نعرف اتجاهه طالما هو يرتفع فهذا جيد خصوصا بعد خسائر ك… https://t.co/7ybA2l1ygP
1429857126059495426 - انا اكبر حاسد يارب اجي فموسم الرياض
934225970 - RT @AJELNEWS24: عاجل 🔴 
.

.
في تميز جديد للوطن .. 🇸🇦🇸🇦

المملكة 

3113638385 - @mohebrahim85 @SaadAldousari @drshamsah • كورونا وباء قاتل &lt;— جملة تسويقية الهدف منها تخويف الناس من الفايروس

• ال… https://t.co/hqAs1XVkOs
1304577000984719360 - RT @gorgeous4ew: عاجل:

ظهور الأسطورة ميسي في إعلان موسم الرياض 😍🇸🇦! https://t.co/tAVuafmXDL
2837083814 - @azxix2020 @Turki_alalshikh مب شرط لان موسم الرياض اللي راح .. كان فيه حضور كبير من الاجانب والخليجين ..
1066123420952420354 - @Abbod200 والله ي عبدالله لو انها كانت فيها اعراض كورونا تستاهل ليش ما تكلمت . خسرو اتنين قراب و من العيله بسبب است… https://t.co/ew7BUFo2lm
2260266832 - رئيس هيئة الترفيه:

يعلن ان موسم الرياض سيتضمن 10 معارض عالمية و350 عرضا مسرحيا و14 منطقة ترفيهية و24 مسرحية عربية… https://t.co/bDbPwV3SBd
178156783 - RT @maralazemi: التمييز بين الطلبة أمر مرفوض وغير مبرر مهما كانت الأسباب وبالتأكيد سوف يكون له تبعات سلبية بين الأطفال..!
1263591648224657409 - ياررب الفرج يارب التوفيق يارب سدد خطانا يارب التمام تكفى يارب ادري انك ما خليتني اجيب درجات بس تكفى بس هالمره خلني… https://t.co/hxH9cSPPQ

1245674671724539907 - RT @SaudiMOH: #الصحة⁩ تعلن عن تسجيل (53) حالة إصابة جديدة بفيروس كورونا (كوفيد-19)، وتسجيل (3) حالات وفيات رحمهم الله، وتسجيل (40) حالة تعا…
1010598389778731009 - RT @BougaryDr: الأخ والصديق العزيز . صاحب حساب #طيار_على_الزمن_باريس . أسئل الله العلي القدير ان يمن عليك بالشفاء العاجل من اصابتك من جائحة…
1358517474308808709 - RT @BougaryDr: الأخ والصديق العزيز . صاحب حساب #طيار_على_الزمن_باريس . أسئل الله العلي القدير ان يمن عليك بالشفاء العاجل من اصابتك من جائحة…
1405487616141803520 - RT @SaudiMOH: #الصحة⁩ تعلن عن تسجيل (53) حالة إصابة جديدة بفيروس كورونا (كوفيد-19)، وتسجيل (3) حالات وفيات رحمهم الله، وتسجيل (40) حالة تعا…
1399630985772339200 - RT @alShama86348964: #الكتلة_الصدرية_عراقية

 لأنهم لبوا نداء الوطن عندما عصفت به جائحة كورونا ، فكانوا بلسماً لجراح المصابين ... فشكرا لهم…
1431683604111822856 - RT @BougaryDr: الأخ والصديق العزيز . صاحب حساب #طيار_على_الزمن_باريس . أسئل الله العلي القدير ان يمن عليك بالشفاء العاجل من اصابتك من جائحة…
1386979466057027584 - 

1356874199764267014 - اههخخخ وربك الكعبه اتراعد❤️❤️❤️😔😔🥺🥺🥺💉💉💉😭😭😭😿😿😿💞💞🤍🤍🐐🐐🐐💕💕💕💗💕
372162621 - RT @imarat_almadina: إحصائية فيروس كورونا في #المدينة_المنورة:
• 2 حالة إصابة جديدة.
• 1 حالة تعافٍ جديدة.
#وزارة_الصحة https://t.co/WtjDXS…
533454902 - رئيس هيئة الترفيه: موسم الرياض يضم 2500 فعالية والدخول مجاناً في 4 مناطق

 https://t.co/S35lkbXKb4

#صحيفة_هام https://t.co/Lyp9l0P5En
535590177 - كل مايجي طاري موسم الرياض قلبي يرفرف
3201993634 - #عاجل ...

#وزارة_الصحة تسجل: 
🔴 53 إصابة جديدة. 
🟢 40 حالة تعافٍ جديدة. 
⚫ 3 حالات وفاة جديدة.

•
#السعودية #الصحة… https://t.co/E21cDtTFN7
2297485541 - شكله موسم ميسي مو موسم الرياض
1366063812374331397 - @Turki_alalshikh @Nefaie_Majed بس احصل فرصة عمل ببلدي ويصير معي فلوس 💵 اجيك واحضر موسم الرياض حقك 👍🏻 اتوقع كلامي واضح
1241457375770542082 - RT @SaudiNews50: الآن..

انطلاق المؤتمر الصحفي لرئيس هيئة الترفيه تركي آل الشيخ للحديث عن موسم الرياض .
1141019521270132739 - مين يصدق اني مجمعه من بدايه 2020 وللحين ماصرفت ولا ريال🫂🫂🫂🫂
420961591 - تكفى خلني اشوف

1354162107353346048 - RT @FCBW_M30: اعلان موسم الرياض برعاية الاسطورة ميسي😍😍😍😍😍 https://t.co/Cs1M2zm6RP
1316401214632984577 - @SPAregions متفائلين جدًا بنجاح موسم الرياض هذي السنة وكل الشكر لجميع القائمين عليه
2429286458 - هذي نهايتك يا ليو 
تطلع سواق في اعلان موسم الرياض https://t.co/lcV9eCWSVg
1401158524470710272 - @XNnA706zHg5fr4l يعني مزال عندي 22 اااا عندك الحق نحي 3 سنين تاع كورونا
249660486 - @Dmoojah قيرفريند 😂😂 وتقول ماني مرتاحه بالرياض ياويله من موسم الرياض شاليللو
352465374 - #من_الخاص

بن تميم تكفى استفسر موسم الرياض فيه خدمة تقسيط من تمارا و تابي ؟
وكم التكلفة لسيدة مع ثلاث بنات علماً با… https://t.co/xHKNkGbaCl
1369674133802483716 - RT @gorgeous4ew: عاجل:

ظهور الأسطورة ميسي في إعلان موسم الرياض 😍🇸🇦! https://t.co/tAVuafmXDL
574858424 - RT @saadjabbar1: خبر لم يأخذ حقه..

عندما اجتاح كورونا القرى الماليزية؛رفع السكان الرايات البيض؛ كعُرف لطلب النجدة ..
فقام جسر جوي سعودي وم…
1061545768853651456 - RT @Team_10_Messi: الأسطورة ميسي في إعلان موسم الرياض 😍 https://t.co/9SvZ3QXH